Study of effect of national factors on home prices in US
Task - Using publically available data for the national factors that impact supply and demand of homes in US, build a model to study the effect of these variables on home prices.



In [ ]:
# importing libraries
import numpy as np
import pandas as pd


In [ ]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CSUSHPISA.csv")

#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

#Selecting data post november 2003
mask = df_CS["DATE"] >= "2003-09-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.head()

(240, 4)


,DATE,CSUSHPISA,Year,Month
0,2003-09-01,136.294,2003,9
1,2003-10-01,137.531,2003,10
2,2003-11-01,138.794,2003,11
3,2003-12-01,140.179,2003,12
4,2004-01-01,141.646,2004,1


In [ ]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([908,909], inplace = True)
print(df_unemp.shape)
df_unemp.tail()

(908, 2)


,DATE,UNRATE
903,2023-04-01,3.4
904,2023-05-01,3.7
905,2023-06-01,3.6
906,2023-07-01,3.5
907,2023-08-01,3.8


In [ ]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("per_capita_gdp.csv", names = ["DATE", "A939RX0Q048SBEA"], skiprows = 1)
print(df_pcgdp.shape)
df_pcgdp.tail()

(307, 2)


,DATE,A939RX0Q048SBEA
302,2022-07-01,65462.0
303,2022-10-01,65783.0
304,2023-01-01,66078.0
305,2023-04-01,66341.0
306,2023-07-01,67039.0


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.


In [ ]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv").drop([830,831])
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(830, 2)


,DATE,FEDFUNDS
825,2023-04-01,4.83
826,2023-05-01,5.06
827,2023-06-01,5.08
828,2023-07-01,5.12
829,2023-08-01,5.33


In [ ]:
# Construction price
df_construction_price = pd.read_csv("construction_price.csv").drop([920])
print(df_construction_price.shape)
df_construction_price.tail()

(920, 2)


,DATE,WPUSI012011
915,2023-04-01,333.366
916,2023-05-01,337.473
917,2023-06-01,337.278
918,2023-07-01,334.449
919,2023-08-01,334.648


In [ ]:
# WORKING_POPULATION
df_WORKING_POPULATION = pd.read_csv("WORKING_POPULATION.csv").drop([])
print(df_WORKING_POPULATION.shape)
df_WORKING_POPULATION.tail()

(560, 2)


,DATE,LFWA64TTUSM647S
555,2023-04-01,2.083833e+08
556,2023-05-01,2.086119e+08
557,2023-06-01,2.087165e+08
558,2023-07-01,2.087998e+08
559,2023-08-01,2.089349e+08


In [ ]:
# Monthly Supply of New Houses in the United States
df_Monthly_Supply = pd.read_csv("Monthly Supply of New Houses in the United States.csv").drop([728])
print(df_Monthly_Supply.shape)
df_Monthly_Supply.tail()

(728, 2)


,DATE,MSACSR
723,2023-04-01,7.6
724,2023-05-01,7.2
725,2023-06-01,7.5
726,2023-07-01,7.0
727,2023-08-01,7.7


In [ ]:
# HOME_residential_loan_rate
df_HOME_residential_loan_rate = pd.read_csv("HOME_residential loan rate.csv").drop([433])
print(df_HOME_residential_loan_rate.shape)
df_HOME_residential_loan_rate.tail()

(433, 2)


,DATE,H8B1027NCBCMG
428,2023-04-01,-0.9
429,2023-05-01,-1.9
430,2023-06-01,-2.8
431,2023-07-01,-4.7
432,2023-08-01,-1.9


In [ ]:
# New Privately-Owned Housing Units Completed: Total Units
df_COMPUTSA = pd.read_csv("COMPUTSA.csv").drop([668])
print(df_COMPUTSA.shape)
df_COMPUTSA.tail()

(668, 2)


,DATE,COMPUTSA
663,2023-04-01,1416.0
664,2023-05-01,1534.0
665,2023-06-01,1492.0
666,2023-07-01,1334.0
667,2023-08-01,1363.0


In [ ]:
# Real Disposable Personal Income
df_DSPIC = pd.read_csv("DSPIC96.csv").drop([776])
print(df_DSPIC.shape)
df_DSPIC.tail()

(776, 2)


,DATE,DSPIC96
771,2023-04-01,16770.5
772,2023-05-01,16829.7
773,2023-06-01,16823.2
774,2023-07-01,16783.7
775,2023-08-01,16764.6


In [ ]:

# Merging Per Capita GDP (Quarterly data)


df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()


,DATE,CSUSHPISA,Year,Month,A939RX0Q048SBEA
0,2003-09-01,136.294,2003,9,NaN
1,2003-10-01,137.531,2003,10,51986.0
2,2003-11-01,138.794,2003,11,NaN
3,2003-12-01,140.179,2003,12,NaN
4,2004-01-01,141.646,2004,1,52179.0


In [ ]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_unemp, df_Fed_rate, df_construction_price, df_WORKING_POPULATION, df_Monthly_Supply, df_HOME_residential_loan_rate , df_COMPUTSA , df_DSPIC ]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.tail()


(920, 12)


,CSUSHPISA,Year,Month,A939RX0Q048SBEA,UNRATE,FEDFUNDS,WPUSI012011,LFWA64TTUSM647S,MSACSR,H8B1027NCBCMG,COMPUTSA,DSPIC96
DATE,,,,,,,,,,,,
2023-04-01,300.198,2023.0,4.0,66341.0,3.4,4.83,333.366,2.083833e+08,7.6,-0.9,1416.0,16770.5
2023-05-01,302.620,2023.0,5.0,NaN,3.7,5.06,337.473,2.086119e+08,7.2,-1.9,1534.0,16829.7
2023-06-01,304.651,2023.0,6.0,NaN,3.6,5.08,337.278,2.087165e+08,7.5,-2.8,1492.0,16823.2
2023-07-01,306.634,2023.0,7.0,67039.0,3.5,5.12,334.449,2.087998e+08,7.0,-4.7,1334.0,16783.7
2023-08-01,309.404,2023.0,8.0,NaN,3.8,5.33,334.648,2.089349e+08,7.7,-1.9,1363.0,16764.6


In [ ]:
df.isna().sum()


CSUSHPISA          680
Year               680
Month              680
A939RX0Q048SBEA    840
UNRATE              12
FEDFUNDS            90
WPUSI012011          0
LFWA64TTUSM647S    360
MSACSR             192
H8B1027NCBCMG      487
COMPUTSA           252
DSPIC96            144
dtype: int64

In [ ]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["A939RX0Q048SBEA"] = df["A939RX0Q048SBEA"].interpolate()


In [ ]:

df.tail()


,CSUSHPISA,Year,Month,A939RX0Q048SBEA,UNRATE,FEDFUNDS,WPUSI012011,LFWA64TTUSM647S,MSACSR,H8B1027NCBCMG,COMPUTSA,DSPIC96
DATE,,,,,,,,,,,,
2023-04-01,300.198,2023.0,4.0,66341.000000,3.4,4.83,333.366,2.083833e+08,7.6,-0.9,1416.0,16770.5
2023-05-01,302.620,2023.0,5.0,66573.666667,3.7,5.06,337.473,2.086119e+08,7.2,-1.9,1534.0,16829.7
2023-06-01,304.651,2023.0,6.0,66806.333333,3.6,5.08,337.278,2.087165e+08,7.5,-2.8,1492.0,16823.2
2023-07-01,306.634,2023.0,7.0,67039.000000,3.5,5.12,334.449,2.087998e+08,7.0,-4.7,1334.0,16783.7
2023-08-01,309.404,2023.0,8.0,67039.000000,3.8,5.33,334.648,2.089349e+08,7.7,-1.9,1363.0,16764.6


In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isna().sum()

CSUSHPISA          0
Year               0
Month              0
A939RX0Q048SBEA    0
UNRATE             0
FEDFUNDS           0
WPUSI012011        0
LFWA64TTUSM647S    0
MSACSR             0
H8B1027NCBCMG      0
COMPUTSA           0
DSPIC96            0
dtype: int64

In [ ]:
df.shape

(239, 12)

In [ ]:
df.head()

,CSUSHPISA,Year,Month,A939RX0Q048SBEA,UNRATE,FEDFUNDS,WPUSI012011,LFWA64TTUSM647S,MSACSR,H8B1027NCBCMG,COMPUTSA,DSPIC96
DATE,,,,,,,,,,,,
2003-10-01,137.531,2003.0,10.0,51986.000000,6.0,1.01,149.6,1.877384e+08,3.8,31.1,1728.0,10982.3
2003-11-01,138.794,2003.0,11.0,52050.333333,5.8,1.00,150.2,1.879721e+08,4.1,29.5,1692.0,11048.4
2003-12-01,140.179,2003.0,12.0,52114.666667,5.7,0.98,149.7,1.881353e+08,4.0,36.1,1716.0,11057.2
2004-01-01,141.646,2004.0,1.0,52179.000000,5.7,1.00,150.0,1.876880e+08,3.8,50.3,1709.0,11051.2
2004-02-01,143.192,2004.0,2.0,52275.666667,5.6,1.01,153.4,1.878755e+08,3.7,37.2,1718.0,11071.0


In [ ]:
df.tail()

,CSUSHPISA,Year,Month,A939RX0Q048SBEA,UNRATE,FEDFUNDS,WPUSI012011,LFWA64TTUSM647S,MSACSR,H8B1027NCBCMG,COMPUTSA,DSPIC96
DATE,,,,,,,,,,,,
2023-04-01,300.198,2023.0,4.0,66341.000000,3.4,4.83,333.366,2.083833e+08,7.6,-0.9,1416.0,16770.5
2023-05-01,302.620,2023.0,5.0,66573.666667,3.7,5.06,337.473,2.086119e+08,7.2,-1.9,1534.0,16829.7
2023-06-01,304.651,2023.0,6.0,66806.333333,3.6,5.08,337.278,2.087165e+08,7.5,-2.8,1492.0,16823.2
2023-07-01,306.634,2023.0,7.0,67039.000000,3.5,5.12,334.449,2.087998e+08,7.0,-4.7,1334.0,16783.7
2023-08-01,309.404,2023.0,8.0,67039.000000,3.8,5.33,334.648,2.089349e+08,7.7,-1.9,1363.0,16764.6


This is our preprocessed datset. Let's save it as "prepared_dataset.csv"

In [ ]:
df.to_csv("prepared_dataset.csv")

In [ ]:
df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
df.head()

,CSUSHPISA,Year,Month,A939RX0Q048SBEA,UNRATE,FEDFUNDS,WPUSI012011,LFWA64TTUSM647S,MSACSR,H8B1027NCBCMG,COMPUTSA,DSPIC96
DATE,,,,,,,,,,,,
2003-10-01,137.531,2003.0,10.0,51986.000000,6.0,1.01,149.6,1.877384e+08,3.8,31.1,1728.0,10982.3
2003-11-01,138.794,2003.0,11.0,52050.333333,5.8,1.00,150.2,1.879721e+08,4.1,29.5,1692.0,11048.4
2003-12-01,140.179,2003.0,12.0,52114.666667,5.7,0.98,149.7,1.881353e+08,4.0,36.1,1716.0,11057.2
2004-01-01,141.646,2004.0,1.0,52179.000000,5.7,1.00,150.0,1.876880e+08,3.8,50.3,1709.0,11051.2
2004-02-01,143.192,2004.0,2.0,52275.666667,5.6,1.01,153.4,1.878755e+08,3.7,37.2,1718.0,11071.0
